# Starting to explore daily train data

In [44]:
import pandas as pd
import numpy as np
import os
import sys
# Hack for using import with relative path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fetchTrainData import fetch_train_data, save_data_to_json


# Fetch undeleted and uncancelled trains for a give date
q = """ 
    {
      trainsByDepartureDate(departureDate: "2020-10-04", 
        take: 10000, 
        where: {
            and: [
                {or: [{deleted: {unequals: null}}, 
                {deleted: {equals: false}}]}, 
                {cancelled: {equals: false}}, 
                {operator: {shortCode: {equals: "vr"}}}
                ]  
        }
      ) 
      {
        trainNumber
        departureDate
        commuterLineid
        deleted
        cancelled
        trainType {
          name
          trainCategory {
            name
          }
        }
        timeTableRows {
          station {
            name
            uicCode
          }
        }
      }
    }"""

In [45]:
result = fetch_train_data(q)

In [46]:
recs = result['data']['trainsByDepartureDate']
df = pd.json_normalize(recs)

In [47]:
result

{'data': {'trainsByDepartureDate': [{'trainNumber': 1,
    'departureDate': '2020-10-04',
    'commuterLineid': '',
    'deleted': None,
    'cancelled': False,
    'trainType': {'name': 'IC', 'trainCategory': {'name': 'Long-distance'}},
    'timeTableRows': [{'station': {'name': 'Helsinki asema', 'uicCode': 1}},
     {'station': {'name': 'Pasila asema', 'uicCode': 10}},
     {'station': {'name': 'Pasila asema', 'uicCode': 10}},
     {'station': {'name': 'Käpylä', 'uicCode': 977}},
     {'station': {'name': 'Käpylä', 'uicCode': 977}},
     {'station': {'name': 'Oulunkylä', 'uicCode': 15}},
     {'station': {'name': 'Oulunkylä', 'uicCode': 15}},
     {'station': {'name': 'Pukinmäki', 'uicCode': 551}},
     {'station': {'name': 'Pukinmäki', 'uicCode': 551}},
     {'station': {'name': 'Malmi', 'uicCode': 17}},
     {'station': {'name': 'Malmi', 'uicCode': 17}},
     {'station': {'name': 'Tapanila', 'uicCode': 552}},
     {'station': {'name': 'Tapanila', 'uicCode': 552}},
     {'station': 

In [48]:
df

,trainNumber,departureDate,commuterLineid,deleted,cancelled,timeTableRows,trainType.name,trainType.trainCategory.name
0,1,2020-10-04,,None,False,"[{'station': {'name': 'Helsinki asema', 'uicCo...",IC,Long-distance
1,3,2020-10-04,,None,False,"[{'station': {'name': 'Helsinki asema', 'uicCo...",IC,Long-distance
2,5,2020-10-04,,None,False,"[{'station': {'name': 'Helsinki asema', 'uicCo...",IC,Long-distance
3,6,2020-10-04,,None,False,"[{'station': {'name': 'Joensuu asema', 'uicCod...",IC,Long-distance
4,7,2020-10-04,,None,False,"[{'station': {'name': 'Helsinki asema', 'uicCo...",S,Long-distance
...,...,...,...,...,...,...,...,...
1139,65880,2020-10-04,,None,False,"[{'station': {'name': 'Jämsänkoski', 'uicCode'...",PAI,Shunting
1140,65937,2020-10-04,,None,False,"[{'station': {'name': 'Kokkola', 'uicCode': 31...",VEV,Shunting
1141,65948,2020-10-04,,None,False,[{'station': {'name': 'Ykspihlaja väliratapiha...,VEV,Shunting
1142,65959,2020-10-04,,None,False,"[{'station': {'name': 'Kokkola', 'uicCode': 31...",VEV,Shunting
